# Setup

In [44]:
!pip install pandas numpy matplotlib seaborn ydata_profiling lightgbm scikit-learn -q

In [5]:
import pandas as pd

raw_data_path =  "/home/nguyen-quang-huy/Dom Technology/datastorm/data/FMCG/raw.csv"
processed_data_path = "/home/nguyen-quang-huy/Dom Technology/datastorm/data/FMCG/processed.csv"

## Utils

In [3]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'])

    return df

# Hierarchy

In [ ]:
df = load_data(processed_data_path)

import json
from collections import defaultdict

# Function to recursively convert sets to lists
def convert_sets_to_lists(obj):
    if isinstance(obj, dict):
        return {k: convert_sets_to_lists(v) for k, v in obj.items()}
    elif isinstance(obj, set):
        return sorted(list(obj))  # sorted for readability & stability
    else:
        return obj

# Initialize nested dictionary
hierarchy = defaultdict(
    lambda: defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(
                lambda: defaultdict(set)
            )
        )
    )
)

# Iterate over rows to build hierarchy
for _, row in df.iterrows():
    city = row["city"]
    channel = row["channel"]
    store = row["store_id"]
    category = row["category"]
    subcategory = row["subcategory"]
    sku = row["sku_id"]

    hierarchy[city][channel][store][category][subcategory].add(sku)


In [31]:
# Convert hierarchy to JSON-friendly structure
hierarchy_json_ready = convert_sets_to_lists(hierarchy)
# Write hierarchy to JSON file
with open("hierarchy.json", "w", encoding="utf-8") as f:
    json.dump(
        hierarchy_json_ready,
        f,
        ensure_ascii=False,
        indent=2
    )


# Preprocessing

In [ ]:
import pandas as pd
import numpy as np

# ==============================================================================
# 1. DEFINE HYBRID LOGIC FUNCTION
# ==============================================================================

def calculate_hybrid_inventory(df):
    """
    Implements Hybrid Inventory Reconstruction logic to correct data inconsistencies.
    
    Logic:
    1. Trust 'Stock-out Flag' (Ground Truth).
    2. Trust System Inflow (Replenishment).
    3. Trust Sales Depletion (avoiding Phantom Inventory).
    
    Returns:
        DataFrame with 'stock_on_hand_hybrid' and 'replenishment_units'.
    """
    # Sort by date to ensure chronological order for mass balance calculation
    df = df.sort_values('date').copy()

    # Calculate raw 'System Inflow' as a reference signal
    # Mass Balance Logic: Inflow = Stock(t) - Stock(t-1) + Sales(t)
    # Note: shift(1) with fillna(0) handles the first record
    prev_sys_stock = df['stock_on_hand'].shift(1).fillna(0)
    system_inflow = df['stock_on_hand'] - prev_sys_stock + df['units_sold']

    # Lists to store the reconstructed values
    new_stocks = []
    new_inflows = []

    # Initialize 'current_stock' using the first system record as an anchor
    # If the first day is already a stock-out, force initialization to 0
    current_stock = df.iloc[0]['stock_on_hand'] if df.iloc[0]['stock_out_flag'] == 0 else 0

    # Iterate through rows efficiently using zip()
    iterator = zip(df['stock_out_flag'], df['stock_on_hand'], df['units_sold'], system_inflow)

    for flag, sys_stock, sales, sys_inflow in iterator:

        replenishment = 0 # Default assumption: No stock arrival

        # --- DECISION LOGIC (TRUST SWITCH) ---

        # CASE A: STOCK-OUT EVENT (Highest Priority)
        # If the flag is raised, force stock to 0 (Ground Truth).
        # We assume physical stock is empty regardless of system record.
        if flag == 1:
            current_stock = 0
            replenishment = 0 

        # CASE B: REPLENISHMENT EVENT (Priority 2)
        # If system shows significant inflow (> 20 units), we TRUST the system record.
        # This resets our calculated stock to match the system's verified count.
        elif sys_inflow > 20: 
            # Calculate actual replenishment needed to reach the new system stock
            # Logic: Inflow = New_System_Stock - Prev_Hybrid_Stock + Sales
            replenishment = max(0, sys_stock - current_stock + sales)

            # Reset hybrid stock to match system stock
            current_stock = sys_stock

        # CASE C: NORMAL SALES / DEPLETION (Priority 3)
        # No stock-out flag and no major inflow -> We TRUST the Sales Logic.
        # We ignore system stock here to avoid 'Phantom Inventory' (stuck high values).
        else:
            replenishment = 0
            # Update stock: New Stock = Old Stock - Sales
            current_stock = max(0, current_stock - sales)

        new_stocks.append(current_stock)
        new_inflows.append(replenishment)

    # Assign results to temporary columns
    df['stock_on_hand_hybrid'] = new_stocks
    df['replenishment_units'] = new_inflows

    return df

# ==============================================================================
# 2. EXECUTION & REPLACEMENT
# ==============================================================================

# Load your raw data
# df = pd.read_csv('your_raw_data.csv') 
# df['date'] = pd.to_datetime(df['date'])

print(">>> Starting Hybrid Inventory Reconstruction...")

# Apply logic per Store-SKU group
# reset_index(drop=True) is essential to flatten the index after groupby
df_clean = df.groupby(['store_id', 'sku_id']).apply(calculate_hybrid_inventory).reset_index(drop=True)

# ---------------------------------------------------------
# Feature Engineering: Create Opening Stock (Critical for ML)
# ---------------------------------------------------------
# Logic: Opening Stock Today = Closing Stock Yesterday
print(">>> Generating 'stock_opening' feature for Machine Learning...")
df_clean['stock_opening'] = df_clean.groupby(['store_id', 'sku_id'])['stock_on_hand_hybrid'].shift(1).fillna(method='bfill')

# ---------------------------------------------------------
# Data Replacement Strategy
# ---------------------------------------------------------
print(">>> Replacing noisy 'stock_on_hand' with corrected hybrid values...")

# 1. Overwrite 'stock_on_hand' with the corrected End-of-Day Hybrid Stock
# Now, 'stock_on_hand' represents the reliable physical count
df_clean['stock_on_hand'] = df_clean['stock_on_hand_hybrid']

# 2. Drop intermediate columns
df_clean.drop(columns=['stock_on_hand_hybrid'], inplace=True)

# ==============================================================================
# 3. SAVE NEW DATASET
# ==============================================================================

output_path = '/home/nguyen-quang-huy/Dom Technology/datastorm/data/FMCG/processed.csv'
df_clean.to_csv(output_path, index=False)

print(f">>> Processing Complete. Clean data saved to: {output_path}")

cols_view = ['date', 'sku_id', 'stock_out_flag', 'units_sold', 'stock_on_hand', 'stock_opening', 'replenishment_units']
df_clean[cols_view].head(35)

>>> Starting Hybrid Inventory Reconstruction...


/tmp/ipykernel_19796/4147786894.py:93: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_clean = df.groupby(['store_id', 'sku_id']).apply(calculate_hybrid_inventory).reset_index(drop=True)


>>> Generating 'stock_opening' feature for Machine Learning...
>>> Replacing noisy 'stock_on_hand' with corrected hybrid values...


/tmp/ipykernel_19796/4147786894.py:100: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_clean['stock_opening'] = df_clean.groupby(['store_id', 'sku_id'])['stock_on_hand_hybrid'].shift(1).fillna(method='bfill')


>>> Processing Complete. Clean data saved to: /home/nguyen-quang-huy/Dom Technology/datastorm/data/FMCG Multi-Country Sales Dataset/processed.csv


,date,sku_id,stock_out_flag,units_sold,stock_on_hand,stock_opening,replenishment_units
0,2021-01-01,SKU0001,0,81,290,290.000,81
1,2021-01-02,SKU0001,0,134,253,290.000,97
2,2021-01-03,SKU0001,0,257,212,253.000,216
3,2021-01-04,SKU0001,0,284,306,212.000,378
4,2021-01-05,SKU0001,0,92,242,306.000,28
5,2021-01-06,SKU0001,0,53,217,242.000,28
6,2021-01-07,SKU0001,0,252,286,217.000,321
7,2021-01-08,SKU0001,0,97,324,286.000,135
8,2021-01-09,SKU0001,0,98,432,324.000,206
9,2021-01-10,SKU0001,1,29,0,432.000,0


In [12]:
df.describe()

,date,year,month,day,weekofyear,weekday,is_weekend,is_holiday,temperature,rain_mm,...,promo_flag,gross_sales,net_sales,stock_on_hand,stock_out_flag,lead_time_days,purchase_cost,margin_pct,replenishment_units,stock_opening
count,1100000,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06,...,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06,1.100000e+06
mean,2022-07-01 20:47:14.181818368,2.022000e+03,6.525613e+00,1.572044e+01,2.657081e+01,3.005479e+00,2.867582e-01,1.369818e-02,1.281501e+01,2.904106e+00,...,8.023364e-02,4.406806e+02,4.299513e+02,3.126905e+02,3.010364e-02,6.500404e+00,4.625546e+00,3.852405e-01,7.124426e+01,3.126729e+02
min,2021-01-01 00:00:00,2.021000e+03,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.800000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,4.900000e-01,-5.000000e-02,0.000000e+00,0.000000e+00
25%,2021-10-01 00:00:00,2.021000e+03,4.000000e+00,8.000000e+00,1.400000e+01,1.000000e+00,0.000000e+00,0.000000e+00,1.061000e+01,1.220000e+00,...,0.000000e+00,1.324400e+02,1.307675e+02,2.730000e+02,0.000000e+00,5.000000e+00,2.400000e+00,3.120000e-01,0.000000e+00,2.730000e+02
50%,2022-07-02 00:00:00,2.022000e+03,7.000000e+00,1.600000e+01,2.700000e+01,3.000000e+00,0.000000e+00,0.000000e+00,1.284000e+01,2.570000e+00,...,0.000000e+00,2.828800e+02,2.778600e+02,3.220000e+02,0.000000e+00,6.000000e+00,4.350000e+00,3.890000e-01,2.900000e+01,3.220000e+02
75%,2023-04-02 00:00:00,2.023000e+03,1.000000e+01,2.300000e+01,4.000000e+01,5.000000e+00,1.000000e+00,0.000000e+00,1.500000e+01,4.150000e+00,...,0.000000e+00,6.052800e+02,5.934600e+02,3.700000e+02,0.000000e+00,8.000000e+00,6.770000e+00,4.690000e-01,1.150000e+02,3.700000e+02
max,2023-12-31 00:00:00,2.023000e+03,1.200000e+01,3.100000e+01,5.300000e+01,6.000000e+00,1.000000e+00,1.000000e+00,2.283000e+01,1.158000e+01,...,1.000000e+00,6.593900e+03,5.144940e+03,6.980000e+02,1.000000e+00,1.700000e+01,1.110000e+01,5.500000e-01,9.800000e+02,6.980000e+02
std,NaN,8.164700e-01,3.447760e+00,8.796262e+00,1.505126e+01,2.000451e+00,4.522478e-01,1.162349e-01,3.371587e+00,2.098997e+00,...,2.716547e-01,4.418005e+02,4.224992e+02,9.490365e+01,1.708726e-01,2.014065e+00,2.662604e+00,1.024581e-01,9.814250e+01,9.490399e+01


In [14]:
zero_count_by_col = (df == 0).sum()
zero_count_by_col

date                         0
year                         0
month                        0
day                          0
weekofyear                   0
weekday                 156713
is_weekend              784566
is_holiday             1084932
temperature                  0
rain_mm                   1005
store_id                     0
country                      0
city                         0
channel                      0
latitude                     0
longitude                    0
sku_id                       0
sku_name                     0
category                     0
subcategory                  0
brand                        0
units_sold                3092
list_price                   0
discount_pct           1011743
promo_flag             1011743
gross_sales               3092
net_sales                 3092
stock_on_hand            48467
stock_out_flag         1066886
lead_time_days               0
supplier_id                  0
purchase_cost                0
margin_p